In [1]:
import numpy
import tensorflow as tf

from ledidi import Ledidi
from ledidi import TensorFlowRegressor

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope

ModuleNotFoundError: No module named 'ledidi'

In [ ]:
# These two objects are necessary for the Basenji model
class GELU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(GELU, self).__init__()
    
    def call(self, x, training):
        return tf.keras.activations.sigmoid(1.702 * x) * x

class StochasticShift(tf.keras.layers.Layer):
    def __init__(self, shift_max=0, pad='uniform', **kwargs):
        super(StochasticShift, self).__init__()

    def call(self, seq_1hot, training):
        return seq_1hot

custom_objects = {
    'StochasticShift': StochasticShift, 
    'GELU': GELU
}

model = load_model("model_human.h5", custom_objects)
model.compile()
regressor = TensorFlowRegressor(model=model)

# Index 687 is CTCF signal in GM12878
mask = numpy.zeros((1, 1024, 5313), dtype='float32')
mask[:, :, 687] = 1.0

mutator = Ledidi(regressor, mask=mask, l=1e2)

sequence = numpy.load("CTCF/CTCF-seqs.npz")['arr_0'].astype('float32')[0].reshape(1, 131072, 4)
epi = model.predict(sequence)

desired_epi = epi.copy()
desired_epi[0, 487:537, 687] = 0.0

edited_sequence = mutator.fit_transform(sequence, desired_epi)
found_epi = model.predict(edited_sequence.astype('float32'))[0, :, 687]